In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
from torch.utils.tensorboard import SummaryWriter
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset
import pandas as pd
import unicodedata
import re
from sklearn.model_selection import train_test_split

In [142]:
with open("data/small_vocab_en.txt", "rt", encoding="unicode-encoding") as f:
    english_text = f.read()

LookupError: unknown encoding: unicode-encoding

In [127]:
english_lines = english_text.strip().split(' .\n')
english_lines = [line for line in english_lines]
english_lines

['new jersey is sometimes quiet during autumn , and it is snowy in april',
 'the united states is usually chilly during july , and it is usually freezing in november',
 'california is usually quiet during march , and it is usually hot in june',
 'the united states is sometimes mild during june , and it is cold in september',
 'your least liked fruit is the grape , but my least liked is the apple',
 'his favorite fruit is the orange , but my favorite is the grape',
 'paris is relaxing during december , but it is usually chilly in july',
 'new jersey is busy during spring , and it is never hot in march',
 'our least liked fruit is the lemon , but my least liked is the grape',
 'the united states is sometimes busy during january , and it is sometimes warm in november',
 'the lime is her least liked fruit , but the banana is my least liked',
 'he saw a old yellow truck',
 'india is rainy during june , and it is sometimes warm in november',
 'that cat was my most loved animal',
 'he dislike

In [129]:
with open("data/small_vocab_fr.txt", "rt", encoding="utf8") as d:
    french_text = d.read()

In [130]:
french_lines = french_text.strip().split(' .\n')
french_lines = [line for line in  french_lines]
french_lines

["new jersey est parfois calme pendant l' automne , et il est neigeux en avril",
 'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre',
 'california est généralement calme en mars , et il est généralement chaud en juin',
 'les états-unis est parfois légère en juin , et il fait froid en septembre',
 'votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme',
 "son fruit préféré est l'orange , mais mon préféré est le raisin",
 'paris est relaxant en décembre , mais il est généralement froid en juillet',
 'new jersey est occupé au printemps , et il est jamais chaude en mars',
 'notre fruit est moins aimé le citron , mais mon moins aimé est le raisin',
 'les états-unis est parfois occupé en janvier , et il est parfois chaud en novembre',
 'la chaux est son moins aimé des fruits , mais la banane est mon moins aimé.\nil a vu un vieux camion jaune',
 'inde est pluvieux en juin , et il est parfois chaud en novembre',
 'ce chat était mon anim

In [131]:
data = pd.DataFrame({"en":english_lines, "fr":french_lines[:len(english_lines)]})
data['eng_len'] = data['en'].str.count(' ')
data['fr_len'] = data['fr'].str.count(' ')
data = data.query('fr_len < 30 & eng_len < 30')
data = data[["en", "fr"]]
data.head()

en  \
0  new jersey is sometimes quiet during autumn , ...   
1  the united states is usually chilly during jul...   
2  california is usually quiet during march , and...   
3  the united states is sometimes mild during jun...   
4  your least liked fruit is the grape , but my l...   

                                                  fr  
0  new jersey est parfois calme pendant l' automn...  
1  les états-unis est généralement froid en juill...  
2  california est généralement calme en mars , et...  
3  les états-unis est parfois légère en juin , et...  
4  votre moins aimé fruit est le raisin , mais mo...

In [132]:
def preprocess_sentence(w):
    w = w.lower().strip()

    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    

    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    return w

In [133]:
data["en"] = data.en.apply(lambda w: preprocess_sentence(w))
data["fr"] = data.fr.apply(lambda w: preprocess_sentence(w))
data.head()

en  \
0  new jersey is sometimes quiet during autumn , ...   
1  the united states is usually chilly during jul...   
2  california is usually quiet during march , and...   
3  the united states is sometimes mild during jun...   
4  your least liked fruit is the grape , but my l...   

                                                  fr  
0  new jersey est parfois calme pendant l automne...  
1  les tats unis est g n ralement froid en juille...  
2  california est g n ralement calme en mars , et...  
3  les tats unis est parfois l g re en juin , et ...  
4  votre moins aim fruit est le raisin , mais mon...

In [134]:
en = spacy.load('en')
fr = spacy.load('fr')


In [135]:
def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]
def tokenize_fr(sentence):
    return [tok.text for tok in fr.tokenizer(sentence)]


In [136]:
EN_TEXT = Field(tokenize=tokenize_en, init_token = "<sos>", eos_token = "<eos>")
FR_TEXT = Field(tokenize=tokenize_fr, init_token = "<sos>", eos_token = "<eos>")

In [137]:
train, test = train_test_split(data, test_size=0.1)
train.to_csv("data/train.csv", index=False)
test.to_csv("data/test.csv", index=False)

In [138]:
data_fields = [('English', EN_TEXT), ('French', FR_TEXT)]
train,test = TabularDataset.splits(path='./data', train='train.csv', validation='test.csv', format='csv', fields=data_fields)

OverflowError: Python int too large to convert to C long

AttributeError: 'DataFrame' object has no attribute 'field_size_limit'